## Data Preprocessing Part

In [1]:
# imports
TESTING = False
import sys
print(sys.executable)
import pandas as pd
from PIL import Image
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
import torch
import torchvision
import torch.optim as optim
import os
from tqdm import tqdm
from torchvision.ops import box_iou
import numpy as np
import matplotlib.pyplot as plt

c:\Users\tiany\anaconda3\python.exe


In [2]:
# check if cuda is available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


In [3]:
# load in data

train_annotation = pd.read_csv('data/train/_annotations.csv')
val_annotation = pd.read_csv('data/valid/_annotations.csv')
test_annotation = pd.read_csv('data/test/_annotations.csv')

if TESTING: #reduce data size to increase testing speed
    train_annotation = train_annotation[:100]

# print the number of samples of the data
print("Number of training, validation and testing samples")
print(len(train_annotation))
print(len(val_annotation))
print(len(test_annotation))

# check the dimension of image is the same
print("min and max width and height of training images: ", train_annotation['width'].min(), train_annotation['width'].max(), train_annotation['height'].min(), train_annotation['height'].max())
print("min and max width and height of validation images: ", val_annotation['width'].min(), val_annotation['width'].max(), val_annotation['height'].min(), val_annotation['height'].max())
print("min and max width and height of testing images: ", test_annotation['width'].min(), test_annotation['width'].max(), test_annotation['height'].min(), test_annotation['height'].max())

Number of training, validation and testing samples
1512
144
72
min and max width and height of training images:  512 512 512 512
min and max width and height of validation images:  512 512 512 512
min and max width and height of testing images:  512 512 512 512


In [4]:
# function map the class name to an integer
def charToVal(c):
    return ord(c) - 64

def valToChar(v):
    return chr(v + 64)

## Dataset Class

In [5]:
# image dimension
img_width = 512
img_height = 512

In [6]:
# create the dataset class

class ASLDataset(Dataset):
    """
    Custom ASL dataset needed for training
    """
    def __init__(self, annotation, img_dir, transforms=None):
        # initialize the annotation, paths to images, and transforms
        self.annotation = annotation
        self.img_dir = img_dir
        self.transforms = transforms

    def __len__(self):
        # return the length of the dataset
        return len(self.annotation)
    
    def __getitem__(self, idx):
        filename = self.annotation.at[idx, 'filename']
        
        class_name = self.annotation.at[idx, 'class']
        xmin = self.annotation.at[idx, 'xmin']
        xmax = self.annotation.at[idx, 'xmax']
        ymin = self.annotation.at[idx, 'ymin']
        ymax = self.annotation.at[idx, 'ymax']

        # read in the image
        path = os.path.join(self.img_dir, filename)
        img = Image.open(path).convert("RGB")

        if self.transforms is not None:
            img = self.transforms(img)

        # target
        target = {}
        target['boxes'] = torch.as_tensor([[xmin, ymin, xmax, ymax]], dtype=torch.float32)
        target['labels'] = torch.as_tensor([charToVal(class_name)], dtype=torch.int64)

        return img, target

In [7]:
# define the transforms
data_transform = transforms.Compose([
    transforms.ToTensor(),
])

# create the dataset
train_dataset = ASLDataset(train_annotation, 'data/train', data_transform)
val_dataset = ASLDataset(val_annotation, 'data/valid', data_transform)
test_dataset = ASLDataset(test_annotation, 'data/test', data_transform)

## Model Training Part

In [8]:
def iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    intersection = max(0, xB - xA + 1) * max(0, yB - yA + 1)

    areaA = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    areaB = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)

    return intersection / float(areaA + areaB - intersection)

In [9]:
def find_best_iou(pred_box, target_box):
    best_iou = 0
    best_index = -1
    for i, box in enumerate(pred_box):
        curr_iou = iou(box, target_box)
        if curr_iou > best_iou:
            best_iou = curr_iou
            best_index = i
    return best_iou, best_index

#test find best iou

target_box = np.array([32., 15., 263., 326.,])
pred_boxes = np.array([[37., 65., 395., 512.], [75.232, 161.32132, 401.234, 486.213]])

best_iou, best_idx = find_best_iou(pred_boxes, target_box)
print(best_iou, best_idx)

0.3423121640133071 0


In [10]:
# collate function
def collate_fn(batch):
    images, targets = zip(*batch)
    images = list(images)
    targets = list(targets)
    return images, targets

In [11]:
# initialize dataloaders
batch_size = 4
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0, collate_fn=collate_fn)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=0, collate_fn=collate_fn)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=0, collate_fn=collate_fn)

In [12]:
# check for possible classes in the dataloader

classes = []
for images, targets in tqdm(train_dataloader):
    for target in targets:
        classes.append(target['labels'][0].item())
classes = set(classes)
print(classes)

100%|██████████| 378/378 [00:10<00:00, 35.24it/s]

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26}


In [13]:
# check the dataloader
images, targets = next(iter(train_dataloader))
print(images)
print(targets)

[tensor([[[0.4627, 0.4078, 0.4196,  ..., 0.7020, 0.6863, 0.6745],
         [0.4824, 0.4275, 0.4235,  ..., 0.6784, 0.6588, 0.6471],
         [0.4980, 0.4392, 0.4275,  ..., 0.6000, 0.5804, 0.5686],
         ...,
         [0.0510, 0.0549, 0.0549,  ..., 0.1804, 0.1804, 0.1804],
         [0.0510, 0.0510, 0.0549,  ..., 0.1804, 0.1804, 0.1804],
         [0.0510, 0.0510, 0.0510,  ..., 0.1804, 0.1804, 0.1804]],

        [[0.3725, 0.3137, 0.3098,  ..., 0.7608, 0.7451, 0.7294],
         [0.3922, 0.3255, 0.3137,  ..., 0.7333, 0.7137, 0.7020],
         [0.4078, 0.3373, 0.3176,  ..., 0.6510, 0.6314, 0.6196],
         ...,
         [0.0275, 0.0314, 0.0314,  ..., 0.0471, 0.0471, 0.0471],
         [0.0275, 0.0275, 0.0314,  ..., 0.0471, 0.0510, 0.0510],
         [0.0275, 0.0275, 0.0275,  ..., 0.0471, 0.0510, 0.0510]],

        [[0.2510, 0.2039, 0.2157,  ..., 0.6471, 0.6314, 0.6275],
         [0.2706, 0.2196, 0.2196,  ..., 0.6353, 0.6157, 0.6118],
         [0.2863, 0.2314, 0.2235,  ..., 0.5843, 0.5647, 0

In [14]:
# define the model
model = fasterrcnn_resnet50_fpn(pretrained=True)
num_classes = 27
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

c:\Users\tiany\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\tiany\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [15]:
# training preparation
model.to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)
epochs = 10

H = {}
H['train_loss'] = []
H['val_ious'] = []
H['val_acc'] = []
H['loss_classifier'] = []
H['loss_box_reg'] = []
H['loss_objectness'] = []
H['loss_rpn_box_reg'] = []

In [16]:
for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    loss_classifier = 0.0
    loss_box_reg = 0.0
    loss_objectness = 0.0
    loss_rpn_box_reg = 0.0
    for images, targets in tqdm(train_dataloader):
        optimizer.zero_grad()
        images = list(image.to(device) for image in images)
        
        
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        #print(len(images))
        #print(len(targets))
        loss_dict = model(images, targets)
        #print('targets: ', targets)
        #print(loss_dict)
        losses = sum(loss for loss in loss_dict.values())

        losses.backward()
        optimizer.step()

        train_loss += losses.item()
        loss_classifier += loss_dict['loss_classifier'].item()
        loss_box_reg += loss_dict['loss_box_reg'].item()
        loss_objectness += loss_dict['loss_objectness'].item()
        loss_rpn_box_reg += loss_dict['loss_rpn_box_reg'].item()
    print(f"Epoch {epoch + 1} Loss: {train_loss/len(train_dataloader)}")
    H["train_loss"].append(train_loss/len(train_dataloader))
    H["loss_classifier"].append(loss_classifier/len(train_dataloader))
    H["loss_box_reg"].append(loss_box_reg/len(train_dataloader))
    H["loss_objectness"].append(loss_objectness/len(train_dataloader))
    H["loss_rpn_box_reg"].append(loss_rpn_box_reg/len(train_dataloader))

    model.eval()
    mean_ious = 0.0
    mean_acc = 0.0
    
    with torch.no_grad():
        for images, targets in tqdm(val_dataloader):
            images = list(image.to(device) for image in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            outputs = model(images)
            #print(outputs)

            ious = []
            iou_idxs = []
            true_classes = []
            pred_classes = []

            for i, output in enumerate(outputs):
                target_box = targets[i]['boxes'][0].cpu().numpy()
                pred_boxes = output['boxes'].cpu().numpy()
                #print(pred_boxes)
                #print(target_box)
                best_iou, best_idx = find_best_iou(pred_boxes, target_box)
                ious.append(best_iou)
                iou_idxs.append(best_idx)

                true_class = targets[i]['labels'][0].cpu().numpy()
                if best_idx == -1:
                    pred_class = 0
                else:
                    pred_class = output['labels'][best_idx].cpu().numpy()
                true_classes.append(true_class)
                pred_classes.append(pred_class)

            mean_ious += sum(ious)/len(ious)
            mean_acc += sum([1 if true_classes[i] == pred_classes[i] else 0 for i in range(len(true_classes))])/len(true_classes)
    H['val_ious'].append(mean_ious/len(val_dataloader))
    H['val_acc'].append(mean_acc/len(val_dataloader))
    print(f"Epoch {epoch + 1} Mean IOU: {mean_ious/len(val_dataloader)}")
    print(f"Epoch {epoch + 1} Mean Accuracy: {mean_acc/len(val_dataloader)}")

100%|██████████| 378/378 [05:01<00:00,  1.25it/s]


Epoch 1 Loss: 0.3071966469682083


100%|██████████| 36/36 [00:14<00:00,  2.47it/s]


Epoch 1 Mean IOU: 0.6981523858105834
Epoch 1 Mean Accuracy: 0.09722222222222222


100%|██████████| 378/378 [05:03<00:00,  1.25it/s]


...

Epoch 7 Loss: 0.2533469888426009


100%|██████████| 36/36 [00:14<00:00,  2.55it/s]


Epoch 7 Mean IOU: 0.8326436718628102
Epoch 7 Mean Accuracy: 0.2222222222222222


  8%|▊         | 30/378 [00:25<04:58,  1.17it/s]


KeyboardInterrupt: 

In [ ]:
# save the model

if not os.path.exists('models'):
    os.makedirs('models')
torch.save(model.state_dict(), 'models/trained_model.pth')

100%|██████████| 25/25 [00:00<00:00, 39.05it/s]

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26}


In [ ]:
# test the model

model.eval()
mean_ious = 0.0
mean_acc = 0.0
overall_true_classes = []
overall_pred_classes = []

with torch.no_grad():
    for images, targets in tqdm(test_dataloader):
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        outputs = model(images)
        #print(outputs)

        ious = []
        iou_idxs = []
        true_classes = []
        pred_classes = []

        for i, output in enumerate(outputs):
            target_box = targets[i]['boxes'][0].cpu().numpy()
            pred_boxes = output['boxes'].cpu().numpy()
            #print(pred_boxes)
            #print(target_box)
            best_iou, best_idx = find_best_iou(pred_boxes, target_box)
            ious.append(best_iou)
            iou_idxs.append(best_idx)

            true_class = targets[i]['labels'][0].cpu().numpy()
            if best_idx == -1:
                pred_class = 0
            else:
                pred_class = output['labels'][best_idx].cpu().numpy()
            true_classes.append(true_class)
            pred_classes.append(pred_class)

        mean_ious += sum(ious)/len(ious)
        mean_acc += sum([1 if true_classes[i] == pred_classes[i] else 0 for i in range(len(true_classes))])/len(true_classes)
        overall_true_classes.extend(true_classes)
        overall_pred_classes.extend(pred_classes)
print(f"Test Mean IOU: {mean_ious/len(test_dataloader)}")
print(f"Test Mean Accuracy: {mean_acc/len(test_dataloader)}")

100%|██████████| 18/18 [00:07<00:00,  2.50it/s]

Test Mean IOU: 0.5485976952218569
Test Mean Accuracy: 0.041666666666666664


In [ ]:
# confusion matrix
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(true_classes, pred_classes)
print(cm)

[[0 0 0 0 0]
 [0 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 1 0 0 0]]


## Plotting

In [ ]:
# training and validation loss

plt.plot(H['train_loss'], label='train_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Training Loss VS Epoch')
plt.savefig('training_loss.png')
plt.show()

In [ ]:
# validation iou

plt.plot(H['val_ious'], label='val_ious')
plt.xlabel('Epoch')
plt.ylabel('IOU')
plt.legend()
plt.title('Validation IOU VS Epoch')
plt.savefig('validation_iou.png')
plt.show()

In [ ]:
# validation accuracy

plt.plot(H['val_acc'], label='val_acc')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Validation Accuracy VS Epoch')
plt.savefig('validation_accuracy.png')
plt.show()

In [ ]:
# 4 loss subplots

plt.subplot(2, 2, 1)
plt.plot(H['loss_classifier'], label='loss_classifier')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Loss Classifier VS Epoch')

plt.subplot(2, 2, 2)
plt.plot(H['loss_box_reg'], label='loss_box_reg')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Loss Box Reg VS Epoch')

plt.subplot(2, 2, 3)
plt.plot(H['loss_objectness'], label='loss_objectness')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Loss Objectness VS Epoch')

plt.subplot(2, 2, 4)
plt.plot(H['loss_rpn_box_reg'], label='loss_rpn_box_reg')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Loss RPN Box Reg VS Epoch')

plt.tight_layout()
plt.savefig('loss_subplots.png')
plt.show()


In [1]:
# directly train a model
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

In [3]:
backbone = torchvision.models.mobilenet_v2(pretrained=False).features
backbone.out_channels = 1280

anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
                                      aspect_ratios=((0.5, 1.0, 2.0),))

roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=[0],
                                                output_size=7,
                                                sampling_ratio=2)
model = FasterRCNN(backbone,
                     num_classes=27,
                     rpn_anchor_generator=anchor_generator,
                     box_roi_pool=roi_pooler)


c:\Users\tiany\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\tiany\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [4]:
# training preparation
model.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
epochs = 10

H = {}
H['train_loss'] = []
H['val_ious'] = []
H['val_acc'] = []
H['loss_classifier'] = []
H['loss_box_reg'] = []
H['loss_objectness'] = []
H['loss_rpn_box_reg'] = []

NameError: name 'device' is not defined

In [ ]:
for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    loss_classifier = 0.0
    loss_box_reg = 0.0
    loss_objectness = 0.0
    loss_rpn_box_reg = 0.0
    for images, targets in tqdm(train_dataloader):
        optimizer.zero_grad()
        images = list(image.to(device) for image in images)
        
        
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        #print(len(images))
        #print(len(targets))
        loss_dict = model(images, targets)
        #print('targets: ', targets)
        #print(loss_dict)
        losses = sum(loss for loss in loss_dict.values())

        losses.backward()
        optimizer.step()

        train_loss += losses.item()
        loss_classifier += loss_dict['loss_classifier'].item()
        loss_box_reg += loss_dict['loss_box_reg'].item()
        loss_objectness += loss_dict['loss_objectness'].item()
        loss_rpn_box_reg += loss_dict['loss_rpn_box_reg'].item()
    print(f"Epoch {epoch + 1} Loss: {train_loss/len(train_dataloader)}")
    H["train_loss"].append(train_loss/len(train_dataloader))
    H["loss_classifier"].append(loss_classifier/len(train_dataloader))
    H["loss_box_reg"].append(loss_box_reg/len(train_dataloader))
    H["loss_objectness"].append(loss_objectness/len(train_dataloader))
    H["loss_rpn_box_reg"].append(loss_rpn_box_reg/len(train_dataloader))

    model.eval()
    mean_ious = 0.0
    mean_acc = 0.0
    
    with torch.no_grad():
        for images, targets in tqdm(val_dataloader):
            images = list(image.to(device) for image in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            outputs = model(images)
            #print(outputs)

            ious = []
            iou_idxs = []
            true_classes = []
            pred_classes = []

            for i, output in enumerate(outputs):
                target_box = targets[i]['boxes'][0].cpu().numpy()
                pred_boxes = output['boxes'].cpu().numpy()
                #print(pred_boxes)
                #print(target_box)
                best_iou, best_idx = find_best_iou(pred_boxes, target_box)
                ious.append(best_iou)
                iou_idxs.append(best_idx)

                true_class = targets[i]['labels'][0].cpu().numpy()
                if best_idx == -1:
                    pred_class = 0
                else:
                    pred_class = output['labels'][best_idx].cpu().numpy()
                true_classes.append(true_class)
                pred_classes.append(pred_class)

            mean_ious += sum(ious)/len(ious)
            mean_acc += sum([1 if true_classes[i] == pred_classes[i] else 0 for i in range(len(true_classes))])/len(true_classes)
    H['val_ious'].append(mean_ious/len(val_dataloader))
    H['val_acc'].append(mean_acc/len(val_dataloader))
    print(f"Epoch {epoch + 1} Mean IOU: {mean_ious/len(val_dataloader)}")
    print(f"Epoch {epoch + 1} Mean Accuracy: {mean_acc/len(val_dataloader)}")

In [ ]:
# save the model

if not os.path.exists('models'):
    os.makedirs('models')
torch.save(model.state_dict(), 'models/trained_model_2.pth')

In [ ]:
# test the model

model.eval()
mean_ious = 0.0
mean_acc = 0.0
overall_true_classes = []
overall_pred_classes = []

with torch.no_grad():
    for images, targets in tqdm(test_dataloader):
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        outputs = model(images)
        #print(outputs)

        ious = []
        iou_idxs = []
        true_classes = []
        pred_classes = []

        for i, output in enumerate(outputs):
            target_box = targets[i]['boxes'][0].cpu().numpy()
            pred_boxes = output['boxes'].cpu().numpy()
            #print(pred_boxes)
            #print(target_box)
            best_iou, best_idx = find_best_iou(pred_boxes, target_box)
            ious.append(best_iou)
            iou_idxs.append(best_idx)

            true_class = targets[i]['labels'][0].cpu().numpy()
            if best_idx == -1:
                pred_class = 0
            else:
                pred_class = output['labels'][best_idx].cpu().numpy()
            true_classes.append(true_class)
            pred_classes.append(pred_class)

        mean_ious += sum(ious)/len(ious)
        mean_acc += sum([1 if true_classes[i] == pred_classes[i] else 0 for i in range(len(true_classes))])/len(true_classes)
        overall_true_classes.extend(true_classes)
        overall_pred_classes.extend(pred_classes)
print(f"Test Mean IOU: {mean_ious/len(test_dataloader)}")
print(f"Test Mean Accuracy: {mean_acc/len(test_dataloader)}")

In [ ]:
# training and validation loss

plt.plot(H['train_loss'], label='train_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Training Loss VS Epoch')
plt.savefig('training_loss.png')
plt.show()

In [ ]:
# validation iou

plt.plot(H['val_ious'], label='val_ious')
plt.xlabel('Epoch')
plt.ylabel('IOU')
plt.legend()
plt.title('Validation IOU VS Epoch')
plt.savefig('validation_iou.png')
plt.show()

In [ ]:
# validation accuracy

plt.plot(H['val_acc'], label='val_acc')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Validation Accuracy VS Epoch')
plt.savefig('validation_accuracy.png')
plt.show()

In [ ]:
# 4 loss subplots

plt.subplot(2, 2, 1)
plt.plot(H['loss_classifier'], label='loss_classifier')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Loss Classifier VS Epoch')

plt.subplot(2, 2, 2)
plt.plot(H['loss_box_reg'], label='loss_box_reg')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Loss Box Reg VS Epoch')

plt.subplot(2, 2, 3)
plt.plot(H['loss_objectness'], label='loss_objectness')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Loss Objectness VS Epoch')

plt.subplot(2, 2, 4)
plt.plot(H['loss_rpn_box_reg'], label='loss_rpn_box_reg')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Loss RPN Box Reg VS Epoch')

plt.tight_layout()
plt.savefig('loss_subplots.png')
plt.show()


## Visualization

In [4]:
torch.cuda.is_available()

False

In [3]:
# load best model
model = torch.load('best-model-parameters.pth')

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.